In [ ]:
#Nota: Este libro se ejecuta cada 24 horas, a las 12 am, para actualizar los precios en Firestore. 
#Este cron esta programado con Papermill: https://filipstollar.medium.com/how-to-schedule-a-jupyter-notebook-to-run-every-day-week-or-month-ae3f992f3afc
#Cron generator: https://crontab-generator.org/

In [ ]:
#import os
#os._exit(00)

In [ ]:
#Se inician Librerias
from urllib.request import Request, urlopen
import json
import copy
import csv
import datetime

#### ApiKey de Tecnolite para servicios 

In [ ]:
#ApiKey de Tecnolite
apiKey = 'povnHJRYuDufMP5i1vJnYNREAShWXgeS'

#### Se inicia Firestore

In [ ]:
#Inicia variables de firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account Voltz DEV
#cred = credentials.Certificate('secrets/clave-servicio-voltz-dev.json')

# Use a service account Voltz Pro
cred = credentials.Certificate('secrets/voltz-pro-key.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()
print('ejecutado')

### Se consulta la lista de codigos de productos en Firestore y se crea la lista de acuerdo a requerimiento de Tecnolite para el servicio

In [ ]:
#Consulta la bd
docs = db.collection(u'products').where(u'source', u'==', 'tecnolite_api').select(field_paths=['sku', 'catalog_code']).stream()
skus = [] 
products = []
for doc in docs:
    skus.append({'MATERIAL' : doc.to_dict()['sku']})
    products.append(doc.to_dict())

print(skus[:3])

###  Se consulta el servicio de Tecnolite con la lista de codigos a consultar

In [1]:
#consulta el servicio de tecnolite de precios con los codigos del catalogo de firestore
body = {"ZB2C_PRECIO_STOCK": {"CLIENTE": "12696"},"MATERIALES": skus}
req = Request('https://tecnolite-preprod.apimanagement.us3.hana.ondemand.com/pro/v1/tecnoliteb2c/products/precio_stock')
req.add_header('ApiKey', apiKey)
req.add_header('content-type', 'application/json')
jsondata = json.dumps(body)
jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
req.add_header('Content-Length', len(jsondataasbytes))
print("Content-Length:", len(jsondataasbytes))
#response = urlopen(req, jsondataasbytes)
#print(response)
content = urlopen(req, jsondataasbytes).read()
json_prices = json.loads(content.decode('utf-8'))
print(json_prices['ZB2C_PRECIO_STOCK.Response']['LISTA_MATERIALES']['item'])
print('Ejecutado.')

NameError: name 'skus' is not defined

### Se actualiza los precios en Firestore

In [ ]:
data = json_prices['ZB2C_PRECIO_STOCK.Response']['LISTA_MATERIALES']['item']
skus = []
for product in products:
    for element in data: 
        if product['sku'] == element['MATERIAL']:
            skus.append({'catalog_code': product['catalog_code'], 'price': element})
            break

batch = db.batch()
batch2 = db.batch()
batch3 = db.batch()
print (data[:2])
i = 0
t = 0
print('total registros a guardar: ', len(data))
for obj in skus:
        element = obj['price']
        sku_price = {
            'suppliers.S2.supplier_code': 'S2', 
            'suppliers.S2.updated_at': datetime.datetime.now(tz=datetime.timezone.utc), 
            'suppliers.S2.supplier_name': 'Tecnolite', 
            'suppliers.S2.arrive_date': None if(element['FECHA_LLEGADA'] == '0000-00-00') else datetime.datetime.strptime(element['FECHA_LLEGADA'], '%Y-%m-%d'),
            'suppliers.S2.stock_transit': float(element['STOCK_TRANSITO']),
            'suppliers.S2.stock': float(element['STOCK_DISPONIBLE']),
            'suppliers.S2.price_best': float(element['PRECIO_NETO']),
            'suppliers.S2.currency': (element['MONEDA'])
        }
        sku_price2 = {
            'product_id': obj['catalog_code'] + '_' + element['MATERIAL'].replace('/', '_'),
            'source': 'tecnolite_api',
            'sku': element['MATERIAL'],
            'supplier_code': 'S2', 
            'updated_at': datetime.datetime.now(tz=datetime.timezone.utc), 
            'supplier_name': 'Tecnolite', 
            'arrive_date': None if(element['FECHA_LLEGADA'] == '0000-00-00') else datetime.datetime.strptime(element['FECHA_LLEGADA'], '%Y-%m-%d'),
            'stock_transit': float(element['STOCK_TRANSITO']),
            'stock': float(element['STOCK_DISPONIBLE']),
            'price_best': float(element['PRECIO_NETO']),
            'currency': (element['MONEDA']),
            'price_margin': 0.25,
        }
        if float(element['STOCK_DISPONIBLE']) > 0:
            sku_price['status'] = 'Disponible'
        else:
            sku_price['status'] = 'No Disponible'
        ref = db.collection(u'products').document(obj['catalog_code'] + '_' + element['MATERIAL'].replace('/', '_'))
        batch.update(ref, sku_price)
        ref2 = db.collection(u'products').document(obj['catalog_code'] + '_' + element['MATERIAL'].replace('/', '_')).collection(u'suppliers').document('S2')
        batch2.set(ref2, sku_price2)
        ref3 = db.collection(u'products').document(obj['catalog_code'] + '_' + element['MATERIAL'].replace('/', '_')).collection(u'history').document()
        batch3.set(ref3, sku_price2)
        t += 1
        if i == 100 or len(data) == t:
            batch.commit()
            batch2.commit()
            batch3.commit()
            i = 0
            batch = db.batch()
            batch2 = db.batch()
            batch3 = db.batch()
            print('guardados: ' + str(t) + ' de ' + str(len(data)))
        else: 
            i += 1
        
print('total de registros guardados', t)

### Revisa productos descontinuados y los pone en stock 0 si ningun otro proveedor tiene stock

In [ ]:
def update_products_without_stock(db, data):
    # Get the products from Firestore where source is 'tecnolite_api'
    products_ref = db.collection(u'products')
    queried_products = products_ref.where(u'source', u'==', 'tecnolite_api').get()
    
    # Convert the data to a set of skus for easier comparison
    local_skus = set(element['MATERIAL'] for element in data)  # Corrected this line

    # Create a batch for updating
    batch = db.batch()

    # Progress indicators
    total_products = len(queried_products)
    processed_products = 0
    updated_to_no_disponible = 0  # Counter for products updated to 'No Disponible'
    print(f'Starting to process {total_products} products')

    # Iterate through the products from Firestore
    for product_snapshot in queried_products:
        product = product_snapshot.to_dict()
        product_sku = product.get('sku')

        # Check if the product is in local data
        if product_sku not in local_skus:
            # Fetch S2 and S3 stock from subcollections
            s2_doc = product_snapshot.reference.collection(u'suppliers').document(u'S2').get().to_dict() or {}
            s3_doc = product_snapshot.reference.collection(u'suppliers').document(u'S3').get().to_dict() or {}

            s2_stock = s2_doc.get('stock', 0)
            s3_stock = s3_doc.get('stock', 0)

            # Check if stock in S2 and S3 is zero
            if s2_stock == 0 and s3_stock == 0:
                # Perform updates on the product
                update_data = {
                    'best_supplier.stock': 0,
                    'status': 'No Disponible',
                    'updated_at': firestore.SERVER_TIMESTAMP  # Add the server timestamp
                }
                # Add the changes to the batch
                batch.update(product_snapshot.reference, update_data)
                updated_to_no_disponible += 1  # Increment the counter

        # Update progress indicators
        processed_products += 1
        print(f'Processed {processed_products} of {total_products} products')

    # Commit the batch
    batch.commit()
    print('Batch update committed')
    print(f'Total products updated to "No Disponible": {updated_to_no_disponible}')  # Print the counter

# Call the update_products_without_stock method
update_products_without_stock(db, data)


### Auditoria de productos que tienen productos con provejal S3 y estan marcados como No Disponible

In [ ]:
def report_products_with_stock(db):
    # Get the products from Firestore where source is 'tecnolite_api'
    products_ref = db.collection(u'products')
    queried_products = products_ref.where(u'source', u'==', 'tecnolite_api').get()
    
    # Progress indicators
    total_products = len(queried_products)
    processed_products = 0
    report_products = []  # List to keep track of products to report
    print(f'Starting to process {total_products} products')

    # Iterate through the products from Firestore
    for product_snapshot in queried_products:
        product = product_snapshot.to_dict()
        product_sku = product.get('sku')
        product_status = product.get('status')

        # Fetch S3 stock from subcollections
        s3_doc = product_snapshot.reference.collection(u'suppliers').document(u'S3').get().to_dict() or {}
        s3_stock = s3_doc.get('stock', 0)

        # Check if stock in S3 is greater than 0 but status is 'No Disponible'
        if s3_stock > 0 and product_status == 'No Disponible':
            # Add product details to report list
            report_products.append({
                'product_id': product_snapshot.id,
                'sku': product_sku,
                'stock_s3': s3_stock,
                'status': product_status
            })

        # Update progress indicators
        processed_products += 1
        print(f'Processed {processed_products} of {total_products} products')

    # Display the report
    print('\nReport:')
    for report_product in report_products:
        print(f"Product ID: {report_product['product_id']}, SKU: {report_product['sku']}, Stock in S3: {report_product['stock_s3']}, Status: {report_product['status']}")

# Call the report_products_with_stock method
report_products_with_stock(db)

In [ ]:
#Consulta la bd
docs = db.collection(u'products').where(u'brand', u'==', u'Tecnolite').select(field_paths=['sku', 'brand']).stream()

for doc in docs:
    p = doc.to_dict()
    p['catalog_code'] = 'C1'
    p['maker_web'] = 'https://tecnolite.mx/'
    p['brand_favicon'] = 'https://firebasestorage.googleapis.com/v0/b/voltz-pro.appspot.com/o/public%2Fbrand_favicons%2FTecnolite.png?alt=media&token=cae931ae-7562-4a3c-8bf5-55e85f71acb9'
    db.collection(u'products').document(doc.id).update(p)
    print('registrado')

print(skus[:3])

In [ ]:
#Consulta la bd
docs = db.collection(u'products').where(u'catalog_code', u'==', u'C1').select(field_paths=['sku']).stream()

for doc in docs:
    p = doc.to_dict()
    db.collection(u'products').document(doc.id).delete()
    print('eliminado')


In [ ]:
ref = db.collection_group(u'suppliers')
docs = ref.stream()
for doc in docs:
    db.document(doc.reference.path).delete()
print('finalizado')